In [1]:
import numpy as np
import pandas as pd
import os

metadata = pd.read_csv("../stock_data_cleaned/stock_vector_and_encoding.csv")

In [2]:
metadata['Symbol']

0          A
1       AACI
2       AADI
3        AAL
4       AAMC
        ... 
4355    ZVSA
4356     ZWS
4357    ZYME
4358    ZYXI
4359    TRUE
Name: Symbol, Length: 4360, dtype: object

In [3]:

data_path = '../stock_data_model_split/'
stack_path = '../stock_data_model_split_stack/'
if not os.path.exists(stack_path):
    os.mkdir(stack_path)
data = pd.read_csv(data_path + 'A' + '.csv')
metadata.loc[metadata['Symbol'] == 'AAPL']

,Symbol,Word_Embedding_Dim1,Word_Embedding_Dim2,Word_Embedding_Dim3,Word_Embedding_Dim4,Word_Embedding_Dim5,Basic Materials,Consumer Discretionary,Consumer Staples,Energy,Finance,Health Care,Industrials,Miscellaneous,Real Estate,Technology,Telecommunications,Utilities
9,AAPL,0.126333,0.063282,-0.129416,-0.024498,-0.179278,0,0,0,0,0,0,0,0,0,1,0,0


In [8]:
data[data['split_0'] != 0].iloc[:, :-5]

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,SMA5,SMA30,...,Stochastic_D,Williams_R,ATR,MFI,OBV,Momentum,ROC,ADX,PLUS_DMI,MINUS_DMI
0,0,2000-08-07,29.059372,29.506437,27.852289,28.433475,24.078014,5560825,28.817954,43.051085,...,3.715464,-96.854310,2.941201,19.487827,-17292283.0,-6.974251,-19.696974,34.275538,13.081110,40.815945
1,1,2000-08-08,28.388769,28.388769,27.315807,27.762877,23.510141,5116121,28.576538,42.185265,...,3.594878,-98.360649,2.810949,19.687914,-22408404.0,-4.247137,-13.268151,35.590893,12.709595,41.019981
2,2,2000-08-09,27.718168,28.522890,27.539343,28.120529,23.813004,4680504,28.460300,41.323915,...,3.836185,-91.549288,2.680420,22.763457,-17727900.0,-2.637697,-8.575583,36.738297,12.733888,39.944854
3,3,2000-08-10,27.941702,28.478184,27.673462,27.896996,23.623709,3123691,28.174177,40.358250,...,4.298353,-93.229157,2.546442,31.026214,-20851591.0,-1.251789,-4.294481,37.803743,12.446449,39.043190
4,4,2000-08-11,28.433475,29.193491,27.852289,29.059372,24.608034,3431251,28.254650,39.517764,...,7.429466,-78.453014,2.460353,21.530072,-17420340.0,-0.447065,-1.515145,38.356802,14.038484,37.522943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1195,2005-05-11,15.629471,15.901288,15.593705,15.879828,13.447346,2972987,15.526467,15.202909,...,87.819290,-1.676004,0.337353,62.568156,-56826622.0,0.693849,4.569008,19.869583,26.115210,18.069179
1196,1196,2005-05-12,15.915594,16.244635,15.500715,15.715308,13.308028,5483795,15.653791,15.197425,...,90.496453,-32.599055,0.366393,71.837170,-62310417.0,0.944206,6.392253,20.630400,29.021348,15.448650
1197,1197,2005-05-13,15.736767,15.944206,15.643777,15.729614,13.320138,4460599,15.723892,15.195518,...,86.291932,-31.718002,0.361681,62.416988,-57849818.0,0.886982,5.975907,21.336873,27.299457,14.532053
1198,1198,2005-05-16,15.629471,15.665236,15.293276,15.422031,13.059673,6348598,15.676681,15.199094,...,75.217988,-50.660775,0.367014,51.855304,-64198416.0,0.422031,2.813543,20.582789,24.981173,20.119462


In [12]:
# 複製相同行數的原始 row
replicated_df = pd.concat([metadata.loc[metadata['Symbol'] == 'AAPL']] * len(data), ignore_index=True)

# 將複製的 rows 添加到股價資料的最前面
combined_df = pd.concat([replicated_df, data.reset_index(drop=True)], axis=1)

In [2]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

# class BlockingTimeSeriesSplit():
#     def __init__(self, n_splits, test_size=0.25):
#         self.n_splits = n_splits
#         self.test_size = test_size
    
#     def get_n_splits(self, X, y, groups):
#         return self.n_splits
    
#     def split(self, X, y=None, groups=None):
#         n_samples = len(X)
#         k_fold_size = n_samples // self.n_splits
#         k_fold_size_x = int(k_fold_size * (1 - self.test_size))
#         indices = np.arange(n_samples)

#         margin = 0
#         for i in range(self.n_splits):
#             start = i * k_fold_size
#             stop = start + k_fold_size
#             mid = k_fold_size_x + start
#             yield indices[start: mid], indices[mid + margin: stop]

# 使用範例
X = np.arange(100)  # 假設有100個樣本
y = np.random.randint(0, 2, size=100)  # 假設對應的標籤

bts = BlockingTimeSeriesSplit(n_splits=5, test_size=0.2)

for train_index, test_index in bts.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("TRAIN:", train_index, "TEST:", test_index)
    print("X_train:", X_train, "X_test:", X_test)
    print("y_train:", y_train, "y_test:", y_test)

TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] TEST: [16 17 18 19]
X_train: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] X_test: [16 17 18 19]
y_train: [1 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0] y_test: [0 0 0 1]
TRAIN: [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35] TEST: [36 37 38 39]
X_train: [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35] X_test: [36 37 38 39]
y_train: [0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 1] y_test: [0 1 1 0]
TRAIN: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55] TEST: [56 57 58 59]
X_train: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55] X_test: [56 57 58 59]
y_train: [1 0 1 0 1 0 0 1 1 1 1 0 1 0 1 0] y_test: [0 1 1 1]
TRAIN: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75] TEST: [76 77 78 79]
X_train: [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75] X_test: [76 77 78 79]
y_train: [1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1] y_test: [1 1 0 1]
TRAIN: [80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95] TEST: [96 97 98 99]
X_train: [80 81 82 83 84 85 86 87 88 89 90 91 9

In [ ]:

X = []
for stockID in metadata['Symbol']:
    data = pd.read_csv(data_path + stockID + '.csv')